In [ ]:
%pip install pandas numpy matplotlib seaborn

# Bitcoin Price and Mining Data Analysis

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

project_root = str(Path().absolute().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data_pipeline.fetch_data import (
    fetch_kibo_mining_data,
    get_full_ohlcv)

from datetime import datetime, timedelta



In [ ]:
#Fetch OHLCV data
btc_ohlcv = get_full_ohlcv()
btc_ohlcv.head()

btc_ohlcv.to_csv("../data/raw/BTC_ohlcv.csv")

# Display first few rows and basic information
print("OHLCV Data Shape:", btc_ohlcv.shape)
btc_ohlcv.head()

In [ ]:
# Get mining metrics
hash_rate = fetch_kibo_mining_data("hash_rate")

difficulty = fetch_kibo_mining_data("difficulty")
        
mining_data = pd.concat([hash_rate, difficulty], axis=1)
mining_data.to_csv("../data/raw/BTC_mining.csv")
        
print("Mining Data Shape:", mining_data.shape)
mining_data.head()

In [4]:
# Loading data
btc_ohlcv = pd.read_csv("../data/raw/BTC_ohlcv.csv", parse_dates=["timestamp"], index_col="timestamp")
mining_data = pd.read_csv("../data/raw/BTC_mining.csv", parse_dates=["timestamp"], index_col="timestamp")
# Merge datasets
full_data = btc_ohlcv.join(mining_data, how="inner")


In [34]:
# 1. Basic Statistics
stats = full_data.describe().T
stats["skewness"] = full_data.skew()
stats["kurtosis"] = full_data.kurtosis()

In [ ]:
# 2. Time Series Analysis
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                   subplot_titles=("Price Action", "Mining Metrics", "Volume"))

# Price Action
fig.add_trace(go.Candlestick(x=full_data.index,
                open=full_data['open'],
                high=full_data['high'],
                low=full_data['low'],
                close=full_data['close'],
                name="OHLC"), row=1, col=1)

# Mining Metrics
fig.add_trace(go.Scatter(x=full_data.index, y=full_data['hash_rate'],
                        name="Hash Rate (EH/s)", line=dict(color='#00CC96')),
                        row=2, col=1)
fig.add_trace(go.Scatter(x=full_data.index, y=full_data['difficulty'],
                        name="Difficulty", line=dict(color='#EF553B')),
                        row=2, col=1)

# Volume
fig.add_trace(go.Bar(x=full_data.index, y=full_data['volume'],
                    name="Volume", marker_color='#636EFA'),
                    row=3, col=1)

fig.update_layout(height=800, showlegend=False)
fig.show()

In [ ]:
# 3. Returns Analysis
full_data['daily_return'] = full_data['close'].pct_change()
full_data['log_return'] = np.log(full_data['close']/full_data['close'].shift(1))

fig = px.histogram(full_data, x="daily_return", 
                  title="Daily Returns Distribution",
                  marginal="box",
                  nbins=100)
fig.show()


In [ ]:
# 4. Volatility Analysis
full_data['volatility'] = full_data['daily_return'].rolling(30).std() * np.sqrt(365)
fig = px.line(full_data, y="volatility", 
             title="30-Day Annualized Volatility")
fig.show()


In [ ]:
# 5. Mining-Price Correlation Analysis
corr_matrix = full_data[['close', 'hash_rate', 'difficulty', 'volume']].corr()
fig = px.imshow(corr_matrix, text_auto=True, 
               title="Feature Correlation Matrix")
fig.show()

In [ ]:
# 6. Advanced Metrics
# Hash Price (USD per EH/day)
full_data['hash_price'] = (full_data['close'] * 6.25 * 144) / (full_data['hash_rate'] * 1e18)
fig = px.line(full_data, y="hash_price", 
             title="Bitcoin Hash Price (USD/EH-day)")
fig.show()

In [35]:
# 7. Seasonality Analysis
daily_returns = full_data['daily_return'].copy()
day_of_week = daily_returns.index.dayofweek
daily_returns.groupby(day_of_week).describe().T




timestamp,0,1,2,3,4,5,6
count,6.000000,6.000000,6.000000,7.000000,7.000000,7.000000,7.000000
mean,0.016039,-0.008507,0.004980,-0.000584,0.009972,-0.001720,-0.013128
std,0.020120,0.034562,0.026465,0.016514,0.020068,0.006434,0.014150
min,-0.006408,-0.051748,-0.023683,-0.026601,-0.023755,-0.016013,-0.031802
25%,0.002696,-0.032190,-0.017225,-0.008074,0.003428,-0.000758,-0.025355
50%,0.010480,-0.011435,0.004354,-0.000458,0.008652,0.000462,-0.006996
75%,0.032714,0.014790,0.022472,0.006715,0.018455,0.001108,-0.001688
max,0.041455,0.039153,0.040765,0.025687,0.041141,0.002807,0.000991


In [51]:
print("Full Data Shape:", full_data.shape)

print(full_data.columns)



full_data = full_data[['open', 'high', 'low', 'close', 'volume', 'hash_rate', 'difficulty']]


full_data.to_csv("../data/raw/full_data.csv")


Full Data Shape: (47, 7)
Index(['open', 'high', 'low', 'close', 'volume', 'hash_rate', 'difficulty'], dtype='object')


In [30]:
def calculate_difficulty_change(data):
    """Estimate upcoming network difficulty change"""
    # Add your difficulty prediction logic here
    return "+5.2%"  # Example value

def generate_forecast_commentary(data):
    """Generate automated insights based on recent trends"""
    last_hash_price = data['hash_price'].iloc[-1]
    price_trend = "bullish" if data['close'].iloc[-1] > data['close'].mean() else "bearish"
    
    commentary = f"""
    <ul>
        <li>Current market sentiment: <span class="{price_trend}">{price_trend}</span></li>
        <li>Miner profitability threshold: ${last_hash_price:.2f}/EH-day</li>
        <li>Next difficulty adjustment estimate: {calculate_difficulty_change(data)}</li>
    </ul>
    """
    return commentary

In [ ]:
report_date = datetime.now().strftime("%Y-%m-%d")
start_date = datetime.now() - timedelta(days=7)
end_date = datetime.now()

report = f"""
    <html>
    <head>
        <title>Weekly Bitcoin Report - {report_date}</title>
        <style>
            body {{ font-family: Arial; margin: 2em; }}
            .metrics {{ display: grid; grid-template-columns: repeat(3, 1fr); gap: 1em; }}
            .card {{ padding: 1em; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }}
            .positive {{ color: #2ecc71; }}
            .negative {{ color: #e74c3c; }}
        </style>
    </head>
    <body>
        <h1>Weekly Bitcoin Market Report ({start_date.date()} - {end_date.date()})</h1>
        
        <div class="metrics">
            <div class="card">
                <h3>📈 Price Performance</h3>
                <p>Weekly Return: {full_data['close'].pct_change().sum():.2%}</p>
                <p>Volatility: {full_data['close'].std():.2f}</p>
            </div>
            
            <div class="card">
                <h3>⛏️ Mining Health</h3>
                <p>Avg Hash Price: ${full_data['hash_price'].mean():.2f}</p>
                <p>Hash Rate Change: {full_data['hash_rate'].pct_change().iloc[-1]:.2%}</p>
            </div>
            
            <div class="card">
                <h3>📊 Market Sentiment</h3>
                <p>Volume Change: {full_data['volume'].pct_change().iloc[-1]:.2%}</p>
            </div>
        </div>

        <h2>📋 Key Statistics</h2>
        {stats.to_html(classes='dataframe')}

        <h2>🔍 Critical Correlations</h2>
        <ul>
            <li>Price-Difficulty: {corr_matrix.loc['close','difficulty']:.2f}</li>
            <li>Price-Hash Rate: {corr_matrix.loc['close','hash_rate']:.2f}</li>
        </ul>

        <h2>📅 Next Week Outlook</h2>
        {generate_forecast_commentary(full_data)}
    </body>
    </html>
    """
    
    # Save report
report_dir = "../reports/weekly/"
os.makedirs(report_dir, exist_ok=True)
filename = f"{report_dir}bitcoin_report_{report_date}.html"
    
with open(filename, "w") as f:
    f.write(report)
        
print(f"Report generated: {filename}")
HTML(report)